In [55]:
# display and plotting imports
%pylab inline 
import seaborn as sns
sns.set()
from IPython.display import SVG

import pandas as pd

# sklearn imports
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

# keras imports
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Input, Embedding, Bidirectional, LSTM

# gensim import for word2vec loading
from gensim.models.keyedvectors import KeyedVectors# display and plotting imports
%pylab inline 
import seaborn as sns
sns.set()
from IPython.display import SVG

import pandas as pd

# sklearn imports
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

# keras imports
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Input, Embedding, Bidirectional, LSTM

# gensim import for word2vec loading
from gensim.models.keyedvectors import KeyedVectors

Populating the interactive namespace from numpy and matplotlib


/Users/mehikapatel/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Plant Classification MVP

This project involves using neural networks and transfer learning to classify leaves by plant type. There are 11 classes total. 

This notebook outlines initial modeling. A simple baseline neural network will be built.

In [ ]:
# display and plotting imports
%pylab inline 
import seaborn as sns
sns.set()
from IPython.display import SVG

import pandas as pd

# sklearn imports
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

# keras imports
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Input, Embedding, Bidirectional, LSTM

# gensim import for word2vec loading
from gensim.models.keyedvectors import KeyedVectors

## Image Processing for Baseline Models

In [126]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/Users/mehikapatel/Plant_NN_Project/data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 150, 150, 3))
    labels = np.zeros(shape=(sample_count,11))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features[i * batch_size : (i + 1) * batch_size] = inputs_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

In [132]:
train_features, train_labels = extract_features(train_dir, 1592)
validation_features, validation_labels = extract_features(validation_dir, 341) 
test_features, test_labels = extract_features(test_dir, 344) 

Found 1592 images belonging to 11 classes.
Found 341 images belonging to 11 classes.
Found 344 images belonging to 11 classes.


In [136]:
#flatten images:
train_features = np.reshape(train_features, (1592, 150 * 150 * 3))
validation_features = np.reshape(validation_features, (341, 150 * 150 * 3))
test_features = np.reshape(test_features, (344, 150 * 150 * 3))

In [137]:
base = keras.Sequential([
    keras.layers.InputLayer(input_shape=train_features.shape[1:]),
    keras.layers.Dense(units=5, activation="relu"),
    keras.layers.Dense(units=11, activation= "softmax"),
])

base.compile("nadam", loss="categorical_crossentropy", metrics=["acc"])

base.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 5)                 337505    
_________________________________________________________________
dense_66 (Dense)             (None, 11)                66        
Total params: 337,571
Trainable params: 337,571
Non-trainable params: 0
_________________________________________________________________


In [138]:
base.fit(train_features, train_labels, epochs=100)

Epoch 1/100
50/50 [==============================] - 2s 9ms/step - loss: 2.4024 - acc: 0.1123
Epoch 2/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3879 - acc: 0.1464
Epoch 3/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3831 - acc: 0.1236
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3761 - acc: 0.1516
Epoch 5/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3739 - acc: 0.1354
Epoch 6/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3665 - acc: 0.1493
Epoch 7/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3676 - acc: 0.1474
Epoch 8/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3602 - acc: 0.1359
Epoch 9/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3628 - acc: 0.1237
Epoch 10/100
50/50 [==============================] - 0s 7ms/step - loss: 2.3585 - acc: 0.1336
Epoch 11/100
50/50 [==============================] - 0s 7m

50/50 [==============================] - 0s 6ms/step - loss: 2.3358 - acc: 0.1415
Epoch 88/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3355 - acc: 0.1378
Epoch 89/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3436 - acc: 0.1322
Epoch 90/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3424 - acc: 0.1449
Epoch 91/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3339 - acc: 0.1523
Epoch 92/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3309 - acc: 0.1386
Epoch 93/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3314 - acc: 0.1460
Epoch 94/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3529 - acc: 0.1362
Epoch 95/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3364 - acc: 0.1359
Epoch 96/100
50/50 [==============================] - 0s 6ms/step - loss: 2.3303 - acc: 0.1454
Epoch 97/100
50/50 [==============================] - 0s 6ms/st

In [140]:
val_loss, val_acc = base.evaluate(validation_features, validation_labels)
print(f'\n\nValidation Accuracy: {val_acc}')

11/11 [==============================] - 0s 4ms/step - loss: 2.3388 - acc: 0.1408


Validation Accuracy: 0.14076246321201324


## Let's try adding more layers to the base model.

In [141]:
bigger = keras.Sequential([
    keras.layers.InputLayer(input_shape=train_features.shape[1:]),
    keras.layers.Dense(units=300, activation="relu"),
    keras.layers.Dense(units=250, activation="relu"),
    keras.layers.Dense(units=200, activation="relu"),
    keras.layers.Dense(units=150, activation="relu"),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dense(units=50, activation="relu"),
    keras.layers.Dense(units=11, activation= "softmax"),
])

bigger.compile("nadam", loss="categorical_crossentropy", metrics=["acc"])

bigger.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 300)               20250300  
_________________________________________________________________
dense_68 (Dense)             (None, 250)               75250     
_________________________________________________________________
dense_69 (Dense)             (None, 200)               50200     
_________________________________________________________________
dense_70 (Dense)             (None, 150)               30150     
_________________________________________________________________
dense_71 (Dense)             (None, 100)               15100     
_________________________________________________________________
dense_72 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_73 (Dense)             (None, 11)              

In [142]:
bigger.fit(train_features, train_labels, epochs=100)

Epoch 1/100
50/50 [==============================] - 18s 292ms/step - loss: 3.7935 - acc: 0.0964
Epoch 2/100
50/50 [==============================] - 14s 274ms/step - loss: 2.2839 - acc: 0.1665
Epoch 3/100
50/50 [==============================] - 13s 252ms/step - loss: 2.1543 - acc: 0.2599
Epoch 4/100
50/50 [==============================] - 13s 257ms/step - loss: 2.0036 - acc: 0.3069
Epoch 5/100
50/50 [==============================] - 13s 254ms/step - loss: 2.1220 - acc: 0.2781
Epoch 6/100
50/50 [==============================] - 13s 253ms/step - loss: 1.8868 - acc: 0.3412
Epoch 7/100
50/50 [==============================] - 13s 254ms/step - loss: 1.6885 - acc: 0.4312
Epoch 8/100
50/50 [==============================] - 13s 253ms/step - loss: 1.4918 - acc: 0.4588
Epoch 9/100
50/50 [==============================] - 13s 259ms/step - loss: 1.4869 - acc: 0.4775
Epoch 10/100
50/50 [==============================] - 13s 256ms/step - loss: 1.3568 - acc: 0.5370
Epoch 11/100
50/50 [=========

50/50 [==============================] - 13s 266ms/step - loss: 0.0185 - acc: 0.9968
Epoch 85/100
50/50 [==============================] - 12s 250ms/step - loss: 0.0265 - acc: 0.9937
Epoch 86/100
50/50 [==============================] - 13s 252ms/step - loss: 0.0176 - acc: 0.9960
Epoch 87/100
50/50 [==============================] - 13s 251ms/step - loss: 0.0185 - acc: 0.9960
Epoch 88/100
50/50 [==============================] - 12s 249ms/step - loss: 0.0071 - acc: 0.9991
Epoch 89/100
50/50 [==============================] - 12s 249ms/step - loss: 0.3034 - acc: 0.9659
Epoch 90/100
50/50 [==============================] - 12s 249ms/step - loss: 1.6768 - acc: 0.3981
Epoch 91/100
50/50 [==============================] - 13s 257ms/step - loss: 0.8784 - acc: 0.7150
Epoch 92/100
50/50 [==============================] - 12s 249ms/step - loss: 0.4882 - acc: 0.8538
Epoch 93/100
50/50 [==============================] - 13s 250ms/step - loss: 0.3795 - acc: 0.8846
Epoch 94/100
50/50 [=============

In [144]:
val_loss, val_acc = base.evaluate(validation_features, validation_labels)
print(f'\n\nValidation Accuracy: {val_acc}')

11/11 [==============================] - 0s 5ms/step - loss: 2.3388 - acc: 0.1408


Validation Accuracy: 0.14076246321201324


## Image Processing for TL Convolutional Networks

In [6]:
#transfer learning model
from tensorflow import keras
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False)
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [25]:
#benchmark function
from sklearn import metrics
def benchmark(model):
    y_pred = model.predict(test_features)
    print(f"accuracy: {metrics.accuracy_score(test_labels, y_pred):,.2f}")


In [8]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/Users/mehikapatel/Plant_NN_Project/data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count,11))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

In [47]:
train_features, train_labels = extract_features(train_dir, 1592)
validation_features, validation_labels = extract_features(validation_dir, 341) 
test_features, test_labels = extract_features(test_dir, 344) 

Found 1592 images belonging to 11 classes.
Found 341 images belonging to 11 classes.
Found 344 images belonging to 11 classes.


In [50]:
#flatten images:
train_features = np.reshape(train_features, (1592, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (341, 4 * 4 * 512))
test_features = np.reshape(test_features, (344, 4 * 4 * 512))

## Baseline Conv NN Model

### Fit our first baseline conv model!

In [51]:
ff_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=train_features.shape[1:]),
    keras.layers.Dense(units=5, activation="relu"),
    keras.layers.Dense(units=11, activation= "softmax"),
])

ff_model.compile("nadam", loss="categorical_crossentropy", metrics=["acc"])

ff_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5)                 40965     
_________________________________________________________________
dense_4 (Dense)              (None, 11)                66        
Total params: 41,031
Trainable params: 41,031
Non-trainable params: 0
_________________________________________________________________


In [53]:
ff_model.fit(train_features, train_labels, epochs=100)

Epoch 1/100
50/50 [==============================] - 0s 3ms/step - loss: 0.9709 - acc: 0.7104
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 0.9492 - acc: 0.7217
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 0.9352 - acc: 0.7173
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.9052 - acc: 0.7544
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.8888 - acc: 0.7557
Epoch 6/100
50/50 [==============================] - 0s 3ms/step - loss: 0.8834 - acc: 0.7582
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 0.8649 - acc: 0.7663
Epoch 8/100
50/50 [==============================] - 0s 2ms/step - loss: 0.8484 - acc: 0.7789
Epoch 9/100
50/50 [==============================] - 0s 2ms/step - loss: 0.8321 - acc: 0.7871
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 0.8275 - acc: 0.7789
Epoch 11/100
50/50 [==============================] - 0s 5m

50/50 [==============================] - 0s 2ms/step - loss: 0.2553 - acc: 0.9711
Epoch 88/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2544 - acc: 0.9730
Epoch 89/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2510 - acc: 0.9692
Epoch 90/100
50/50 [==============================] - 0s 3ms/step - loss: 0.2506 - acc: 0.9724
Epoch 91/100
50/50 [==============================] - 0s 3ms/step - loss: 0.2463 - acc: 0.9705
Epoch 92/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2405 - acc: 0.9698
Epoch 93/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2387 - acc: 0.9730
Epoch 94/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2380 - acc: 0.9717
Epoch 95/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2332 - acc: 0.9749
Epoch 96/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2278 - acc: 0.9742
Epoch 97/100
50/50 [==============================] - 0s 2ms/st

In [83]:
val_loss, val_acc = ff_model.evaluate(validation_features, validation_labels)
print(f'\n\nValidation Accuracy: {val_acc}')

11/11 [==============================] - 0s 2ms/step - loss: 2.6376 - acc: 0.2082


Validation Accuracy: 0.20821113884449005


### We will need a lot of modifications to improve this model on our validation data!

## Modified Models

**More Layers & Nodes**

In [150]:
base_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=train_features.shape[1:]),
    keras.layers.Dense(units=150, activation="relu"),
    keras.layers.Dense(units=125, activation="relu"),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dense(units=75, activation="relu"),
    keras.layers.Dense(units=50, activation="relu"),
    keras.layers.Dense(units=25, activation="relu"),
    keras.layers.Dense(units=11, activation= "softmax"),
])

base_model.compile("nadam", loss="categorical_crossentropy", metrics=["acc"])

base_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 150)               10125150  
_________________________________________________________________
dense_76 (Dense)             (None, 125)               18875     
_________________________________________________________________
dense_77 (Dense)             (None, 100)               12600     
_________________________________________________________________
dense_78 (Dense)             (None, 75)                7575      
_________________________________________________________________
dense_79 (Dense)             (None, 50)                3800      
_________________________________________________________________
dense_80 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_81 (Dense)             (None, 11)              

In [151]:
base_model.fit(train_features, train_labels, epochs=100,callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True)
    ])

Epoch 1/100
50/50 [==============================] - 10s 148ms/step - loss: 2.8971 - acc: 0.1125
Epoch 2/100
50/50 [==============================] - 9s 173ms/step - loss: 2.3068 - acc: 0.1779
Epoch 3/100
50/50 [==============================] - 6s 108ms/step - loss: 2.2817 - acc: 0.1666
Epoch 4/100
50/50 [==============================] - 6s 111ms/step - loss: 2.2637 - acc: 0.1694
Epoch 5/100
50/50 [==============================] - 6s 111ms/step - loss: 2.3900 - acc: 0.1184
Epoch 6/100
50/50 [==============================] - 6s 125ms/step - loss: 2.3827 - acc: 0.1198
Epoch 7/100
50/50 [==============================] - 8s 155ms/step - loss: 2.3782 - acc: 0.1097
Epoch 8/100
50/50 [==============================] - 6s 122ms/step - loss: 2.3705 - acc: 0.1347
Epoch 9/100
50/50 [==============================] - 7s 133ms/step - loss: 2.3679 - acc: 0.1448
Epoch 10/100
50/50 [==============================] - 5s 105ms/step - loss: 2.3668 - acc: 0.1415
Epoch 11/100
50/50 [==================

KeyboardInterrupt: 

In [96]:
val_loss, val_acc = base_model.evaluate(validation_features, validation_labels)
print(f'\n\nValidation Accuracy: {val_acc}')

11/11 [==============================] - 0s 4ms/step - loss: 0.6429 - acc: 0.8856


Validation Accuracy: 0.8856304883956909


##### This model did a lot better!!!

**More Layers + Regularization to account for lower accuracy in validation versus training**

In [110]:
third_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=train_features.shape[1:]),
    keras.layers.Dense(units=150, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=125, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=75, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=50, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=25, activation="relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=11, activation= "softmax"),
])

third_model.compile("nadam", loss="categorical_crossentropy", metrics=["acc"])

third_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 150)               1228950   
_________________________________________________________________
dropout_15 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 125)               18875     
_________________________________________________________________
dropout_16 (Dropout)         (None, 125)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 100)               12600     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 75)               

In [111]:
third_model.fit(train_features, train_labels, epochs=22)

Epoch 1/22
50/50 [==============================] - 2s 12ms/step - loss: 2.2376 - acc: 0.1863
Epoch 2/22
50/50 [==============================] - 1s 12ms/step - loss: 1.3760 - acc: 0.5375
Epoch 3/22
50/50 [==============================] - 1s 13ms/step - loss: 0.8551 - acc: 0.7287
Epoch 4/22
50/50 [==============================] - 1s 13ms/step - loss: 0.5546 - acc: 0.8364
Epoch 5/22
50/50 [==============================] - 1s 12ms/step - loss: 0.4216 - acc: 0.8599
Epoch 6/22
50/50 [==============================] - 1s 12ms/step - loss: 0.2896 - acc: 0.9127
Epoch 7/22
50/50 [==============================] - 1s 13ms/step - loss: 0.2678 - acc: 0.9057
Epoch 8/22
50/50 [==============================] - 1s 14ms/step - loss: 0.1543 - acc: 0.9606
Epoch 9/22
50/50 [==============================] - 1s 13ms/step - loss: 0.2377 - acc: 0.9299
Epoch 10/22
50/50 [==============================] - 1s 13ms/step - loss: 0.1406 - acc: 0.9602
Epoch 11/22
50/50 [==============================] - 1s 20m

In [112]:
val_loss, val_acc = third_model.evaluate(validation_features, validation_labels)
print(f'\n\nValidation Accuracy: {val_acc}')

11/11 [==============================] - 0s 4ms/step - loss: 0.7351 - acc: 0.8680


Validation Accuracy: 0.8680351972579956


##### The dropout reg model did not do as well so we will stick with our other model. 